In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# importing required modules 
from zipfile import ZipFile 
  
# specifying the zip file name 
filename = "Data/titanic.zip"
  
# opening the zip file in READ mode 
with ZipFile(filename, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
    print()
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall(path = f'./{filename[:-4]}') 
    print('Done!') 

File Name                                             Modified             Size
gender_submission.csv                          2019-12-11 02:17:12         3258
test.csv                                       2019-12-11 02:17:12        28629
train.csv                                      2019-12-11 02:17:12        61194

Extracting all the files now...
Done!


In [35]:
df = pd.read_csv(f'{filename[:-4]}/train.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [37]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [38]:
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.preprocessing import RobustScaler,FunctionTransformer,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [39]:
X = df.drop('Survived',axis = 1)
y = df['Survived']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify = y)
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
692,693,3,"Lam, Mr. Ali",male,NaN,0,0,1601,56.4958,NaN,S
481,482,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0000,NaN,S
527,528,1,"Farthing, Mr. John",male,NaN,0,0,PC 17483,221.7792,C95,S
855,856,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.0,0,1,392091,9.3500,NaN,S
801,802,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
359,360,3,"Mockler, Miss. Helen Mary ""Ellie""",female,NaN,0,0,330980,7.8792,NaN,Q
258,259,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C
736,737,3,"Ford, Mrs. Edward (Margaret Ann Watson)",female,48.0,1,3,W./C. 6608,34.3750,NaN,S
462,463,1,"Gee, Mr. Arthur H",male,47.0,0,0,111320,38.5000,E63,S


In [41]:
CT = ColumnTransformer([
    ('set_index',FunctionTransformer(lambda df : df.set_index('PassengerId'),validate = False),['PassengerId']),
    ('drop_cols',"drop",['Name','Ticket','Cabin']),
    ('numerical_passthrough',"passthrough",['Pclass','SibSp','Parch','Fare']),
    ('categorical_encoder',OrdinalEncoder(),['Sex','Embarked']),
    ])

In [109]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6,input_shape = (6,),activation = 'relu'),
    tf.keras.layers.Dense(10,activation = 'relu'),
    tf.keras.layers.Dense(20,activation = 'relu'),
    tf.keras.layers.Dense(50,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
             metrics = ['accuracy'])

In [110]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 6)                 42        
                                                                 
 dense_61 (Dense)            (None, 10)                70        
                                                                 
 dense_62 (Dense)            (None, 20)                220       
                                                                 
 dense_63 (Dense)            (None, 50)                1050      
                                                                 
 dense_64 (Dense)            (None, 1)                 51        
                                                                 
Total params: 1433 (5.60 KB)
Trainable params: 1433 (5.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
pipe = Pipeline([
    ('ColumnTransformer',CT),
    ('Fix_missing_values',SimpleImputer(strategy = 'most_frequent')),
    ('Scaling',RobustScaler()),
])

final_X_train = pipe.fit_transform(X_train,y_train)
final_X_test = pipe.transform(X_test)

In [112]:
model.fit(final_X_train,y_train,epochs = 30,validation_data = (final_X_test,y_test),validation_steps = len(final_X_test))

Epoch 1/50
23/23 [==============================] - 2s 15ms/step - loss: 0.5557 - accuracy: 0.7500 - val_loss: 0.5206 - val_accuracy: 0.7542
Epoch 2/50
23/23 [==============================] - 0s 11ms/step - loss: 0.4590 - accuracy: 0.8020 - val_loss: 0.4885 - val_accuracy: 0.7765
Epoch 3/50
23/23 [==============================] - 0s 10ms/step - loss: 0.4432 - accuracy: 0.8118 - val_loss: 0.4965 - val_accuracy: 0.7542
Epoch 4/50
23/23 [==============================] - 0s 10ms/step - loss: 0.4442 - accuracy: 0.8132 - val_loss: 0.4831 - val_accuracy: 0.7989
Epoch 5/50
23/23 [==============================] - 0s 10ms/step - loss: 0.4337 - accuracy: 0.8062 - val_loss: 0.4665 - val_accuracy: 0.7989
Epoch 6/50
23/23 [==============================] - 0s 9ms/step - loss: 0.4281 - accuracy: 0.8160 - val_loss: 0.4807 - val_accuracy: 0.7877
Epoch 7/50
23/23 [==============================] - 0s 13ms/step - loss: 0.4324 - accuracy: 0.8048 - val_loss: 0.4636 - val_accuracy: 0.7821
Epoch 8/50
23/

In [113]:
preds = tf.round(model.predict(final_X_test))
print(classification_report(y_test,preds))

6/6 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.78      0.90      0.84       110
           1       0.79      0.59      0.68        69

    accuracy                           0.78       179
   macro avg       0.78      0.75      0.76       179
weighted avg       0.78      0.78      0.77       179

